In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv("cancer.csv")

In [5]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [9]:
df=pd.get_dummies(df,columns=['diagnosis'],drop_first=True)

In [12]:
dtree=DecisionTreeClassifier(criterion='entropy',max_depth=6,random_state=0)
dtree.fit(x_train,y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=6, random_state=0)

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df['Unnamed: 32'].unique()

In [3]:
df=df.drop(['Unnamed: 32'],axis=1)

In [4]:
df=df.drop(['id'],axis=1)

In [ ]:
df['diagnosis'].unique()   #M-->patient has cancer   #B---->no cancer

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [10]:
x = df.drop(["diagnosis_M"], axis = 1)
y=df['diagnosis_M']

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=1)

In [13]:
dtree=DecisionTreeClassifier(criterion='entropy',max_depth=6,random_state=0)
dtree.fit(x_train,y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=6, random_state=0)

In [14]:
#without treating outliers
print(dtree.score(x_train,y_train))
print(dtree.score(x_test,y_test))

1.0
0.9064327485380117


In [ ]:
plt.subplots(figsize=(20,17))
sns.heatmap(df.corr(),annot=True)

In [ ]:
sns.boxplot('radius_mean',data=df)

In [ ]:
#fig,A=plt.subplots(2,2)
plt.subplot(2,2,1)
sns.boxplot('radius_mean',data=df)
plt.title('radiusmean')
plt.subplot(2,2,2)
sns.boxplot('texture_mean',data=df)
plt.title('TextureMean')
plt.subplot(2,2,3)
sns.boxplot('perimeter_mean',data=df)
plt.subplot(2,2,4)
sns.boxplot('area_mean',data=df)
#fig.show()

In [ ]:
plt.hist('texture_mean',data=df,color='orchid')

In [ ]:
texture_mean=df['texture_mean']

In [ ]:
outliers = []
def detect_outliers_iqr(data):
    data = sorted(data)
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    # print(q1, q3)
    IQR = q3-q1
    lwr_bound = q1-(1.5*IQR)
    upr_bound = q3+(1.5*IQR)
    # print(lwr_bound, upr_bound)
    for i in data: 
        if (i<lwr_bound or i>upr_bound):
            outliers.append(i)
    return outliers# Driver code
sample_outliers = detect_outliers_iqr(texture_mean)
print("Outliers from IQR method: ", sample_outliers)

In [ ]:
tenth_percentile = np.percentile(texture_mean, 10)
ninetieth_percentile = np.percentile(texture_mean, 90)
# print(tenth_percentile, ninetieth_percentile)
b = np.where(texture_mean<tenth_percentile, tenth_percentile, texture_mean)
b = np.where(b>ninetieth_percentile, ninetieth_percentile, b)
# print("Sample:", sample)
#print("New array:",b)

In [ ]:
df['texture_mean']=b

In [ ]:
area_mean=df['area_mean']

In [ ]:
tenth_percentile = np.percentile(area_mean, 10)
ninetieth_percentile = np.percentile(area_mean, 90)
# print(tenth_percentile, ninetieth_percentile)
b = np.where(area_mean<tenth_percentile, tenth_percentile, area_mean)
b = np.where(b>ninetieth_percentile, ninetieth_percentile, b)
# print("Sample:", sample)
#print("New array:",b)

In [ ]:
df['area_mean']=b

In [ ]:
cols = ['radius_mean','perimeter_mean'] # one or more

Q1 = df[cols].quantile(0.10)
Q3 = df[cols].quantile(0.90)
IQR = Q3 - Q1

df = df[~((df[cols] < (Q1 - 1.5 * IQR)) |(df[cols] > (Q3 + 1.5 * IQR))).any(axis=1)]

In [ ]:
df.columns


In [ ]:
plt.subplot(2,2,1)
sns.boxplot('smoothness_mean',data=df,color='orchid')
plt.title('Smoothness_mean')
plt.subplot(2,2,2)
sns.boxplot('compactness_mean',data=df,color='y')
plt.title('compactnessmean')
plt.subplot(2,2,3)
sns.boxplot('concavity_mean',data=df,color='g')
plt.subplot(2,2,4)
sns.boxplot('concave points_mean',data=df)

In [ ]:
cols = ['concavity_mean','concave points_mean','smoothness_mean','compactness_mean'] # one or more

Q1 = df[cols].quantile(0.25)
Q3 = df[cols].quantile(0.75)
IQR = Q3 - Q1

df = df[~((df[cols] < (Q1 - 1.5 * IQR)) |(df[cols] > (Q3 + 1.5 * IQR))).any(axis=1)]

In [ ]:
sns.distplot(df['symmetry_mean'])

In [ ]:
cols=['radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst']

In [ ]:
Q1 = df[cols].quantile(0.10)
Q3 = df[cols].quantile(0.90)
IQR = Q3 - Q1

df = df[~((df[cols] < (Q1 - 1.5 * IQR)) |(df[cols] > (Q3 + 1.5 * IQR))).any(axis=1)]

In [ ]:
df.head()

In [ ]:
x = df.drop(["diagnosis_M"], axis = 1)
y=df['diagnosis_M']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=1)

In [ ]:
dtree=DecisionTreeClassifier(criterion='entropy',max_depth=6,random_state=0)
dtree.fit(x_train,y_train)

In [ ]:
print(dtree.score(x_train,y_train))
print(dtree.score(x_test,y_test))

In [ ]:
from sklearn import metrics

In [ ]:
#confusion metrics

y_predict = dtree.predict(x_test)
print(dtree.score(x_test, y_test))
cm=metrics.confusion_matrix(y_test, y_predict,labels=[0, 1])

df_cm = pd.DataFrame(cm, index = [i for i in ["No","Yes"]],
                  columns = [i for i in ["No","Yes"]])
plt.figure(figsize = (7,5))
sns.heatmap(df_cm, annot=True ,fmt='g')

In [ ]:
#from sklearn import tree

In [ ]:


from sklearn.ensemble import BaggingClassifier
bgcl = BaggingClassifier( n_estimators=100,base_estimator=dtree,random_state=1)
bgcl = bgcl.fit(x_train, y_train)
y_predict = bgcl.predict(x_test)
print(bgcl.score(x_train,y_train))
print(bgcl.score(x_test , y_test))

In [ ]:
#confusion metrics

y_predict = bgcl.predict(x_test)
print(bgcl.score(x_test, y_test))
cm=metrics.confusion_matrix(y_test, y_predict,labels=[0, 1])

df_cm = pd.DataFrame(cm, index = [i for i in ["No","Yes"]],
                  columns = [i for i in ["No","Yes"]])
plt.figure(figsize = (7,5))
sns.heatmap(df_cm, annot=True ,fmt='g')

In [ ]:
##bagging without decision tree

from sklearn.ensemble import BaggingClassifier
bgcl = BaggingClassifier( n_estimators=70,random_state=0)
bgcl = bgcl.fit(x_train, y_train)
y_predict = bgcl.predict(x_test)
print(bgcl.score(x_train,y_train))
print(bgcl.score(x_test , y_test))

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
abcl = AdaBoostClassifier(n_estimators=100, random_state=1)
abcl = abcl.fit(x_train, y_train)
y_predict = abcl.predict(x_test)
print(abcl.score(x_train,y_train))
print(abcl.score(x_test , y_test))

In [ ]:
#confusion metrics

y_predict = abcl.predict(x_test)
print(abcl.score(x_test, y_test))
cm=metrics.confusion_matrix(y_test, y_predict,labels=[0, 1])

df_cm = pd.DataFrame(cm, index = [i for i in ["No","Yes"]],
                  columns = [i for i in ["No","Yes"]])
plt.figure(figsize = (7,5))
sns.heatmap(df_cm, annot=True ,fmt='g')

In [ ]:
from sklearn.ensemble import RandomForestRegressor
RandomForst = RandomForestRegressor(n_estimators = 500, random_state = 0,max_features=20)
RandomForst.fit(x_train, y_train)
RandomForst.score(x_train,y_train)
RandomForst.fit(x_test, y_test)
print(RandomForst.score(x_train,y_train))
print(RandomForst.score(x_test,y_test))

In [ ]:
y_predict = RandomForst.predict(x_test)

In [ ]:
#confusion metrics

#y_predict = RandomForst.predict(x_test)
#print(RandomForst.score(x_test, y_test))
cm=metrics.confusion_matrix(y_test, y_predict,labels=[0, 1])

df_cm = pd.DataFrame(cm, index = [i for i in ["No","Yes"]],
                  columns = [i for i in ["No","Yes"]])
plt.figure(figsize = (7,5))
sns.heatmap(df_cm, annot=True ,fmt='g')

In [ ]:
#gradient boosting

from sklearn.ensemble import GradientBoostingClassifier
gbcl = GradientBoostingClassifier(n_estimators = 100,random_state=1,max_features=25)
gbcl = gbcl.fit(x_train, y_train)
y_predict = gbcl.predict(x_test)
print(gbcl.score(x_train,y_train))
print(gbcl.score(x_test, y_test))

In [ ]:
y_predict = gbcl.predict(x_test)
print(gbcl.score(x_test, y_test))
cm=metrics.confusion_matrix(y_test, y_predict,labels=[0, 1])

df_cm = pd.DataFrame(cm, index = [i for i in ["No","Yes"]],
                  columns = [i for i in ["No","Yes"]])
plt.figure(figsize = (7,5))
sns.heatmap(df_cm, annot=True ,fmt='g')